# Azure OpenAI

:::caution
您当前正在浏览关于如何使用 Azure OpenAI [文本补全模型](/docs/concepts/text_llms)的文档。最新且最受欢迎的 Azure OpenAI 模型是 [聊天补全模型](/docs/concepts/chat_models)。

除非您专门使用 `gpt-3.5-turbo-instruct`，否则您很可能是在查找[此页面](/docs/integrations/chat/azure_chat_openai/)。
:::

此页面将介绍如何将 LangChain 与 [Azure OpenAI](https://aka.ms/azure-openai) 结合使用。

Azure OpenAI API 与 OpenAI 的 API 兼容。`openai` Python 包可以轻松地同时使用 OpenAI 和 Azure OpenAI。您可以像调用 OpenAI 一样调用 Azure OpenAI，但需注意以下几点例外。

## API 配置
您可以通过环境变量配置 `openai` 包以使用 Azure OpenAI。以下是 `bash` 的配置示例：

```bash
# 您想使用的 API 版本：设置为 `2023-12-01-preview` 以使用已发布的版本。
export OPENAI_API_VERSION=2023-12-01-preview
# 您的 Azure OpenAI 资源的基 URL。您可以在 Azure 门户中找到您的 Azure OpenAI 资源下的此信息。
export AZURE_OPENAI_ENDPOINT=https://your-resource-name.openai.azure.com
# 您的 Azure OpenAI 资源的 API 密钥。您可以在 Azure 门户中找到您的 Azure OpenAI 资源下的此信息。
export AZURE_OPENAI_API_KEY=<your Azure OpenAI API key>
```

或者，您可以在运行的 Python 环境中直接配置 API：

```python
import os
os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
```

## Azure Active Directory 身份验证
您可以通过两种方式向 Azure OpenAI 进行身份验证：
- API 密钥
- Azure Active Directory (AAD)

使用 API 密钥是开始的最简单方法。您可以在 Azure 门户中找到您的 Azure OpenAI 资源的 API 密钥。

但是，如果您有复杂的安全需求，则可能需要使用 Azure Active Directory。您可以在[此处](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/managed-identity)找到有关如何将 AAD 与 Azure OpenAI 结合使用的更多信息。

如果您在本地开发，则需要安装 Azure CLI 并登录。您可以在[此处](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli)安装 Azure CLI。然后，运行 `az login` 进行登录。

为您的 Azure OpenAI 资源范围内的 Azure 角色分配 `Cognitive Services OpenAI User` 的角色。这将允许您从 AAD 获取令牌以与 Azure OpenAI 一起使用。您可以将此角色分配给用户、组、服务主体或托管标识。有关 Azure OpenAI RBAC 角色的更多信息，请参见[此处](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/role-based-access-control)。

要在 Python 中使用 AAD 和 LangChain，请安装 `azure-identity` 包。然后，将 `OPENAI_API_TYPE` 设置为 `azure_ad`。接下来，使用 `DefaultAzureCredential` 类通过调用 `get_token` 从 AAD 获取令牌，如下所示。最后，将 `OPENAI_API_KEY` 环境变量设置为令牌值。

```python
import os
from azure.identity import DefaultAzureCredential

# 获取 Azure 凭据
credential = DefaultAzureCredential()

# 将 API 类型设置为 `azure_ad`
os.environ["OPENAI_API_TYPE"] = "azure_ad"
# 将 API_KEY 设置为来自 Azure 凭据的令牌
os.environ["OPENAI_API_KEY"] = credential.get_token("https://cognitiveservices.azure.com/.default").token
```

`DefaultAzureCredential` 类是开始使用 AAD 身份验证的简便方法。您也可以根据需要自定义凭据链。在下面的示例中，我们首先尝试托管标识，然后回退到 Azure CLI。如果您在 Azure 中运行代码，但希望在本地进行开发，这将非常有用。

```python
from azure.identity import ChainedTokenCredential, ManagedIdentityCredential, AzureCliCredential

credential = ChainedTokenCredential(
    ManagedIdentityCredential(),
    AzureCliCredential()
)
```

## 部署
使用 Azure OpenAI 时，您可以设置常见 GPT-3 和 Codex 模型的部署。调用 API 时，需要指定要使用的部署。

_**注意**：本文档适用于 Azure 文本补全模型。GPT-4 等模型是聊天模型。它们具有略微不同的接口，可以通过 `AzureChatOpenAI` 类访问。有关 Azure 聊天模型的文档，请参阅[Azure 聊天 OpenAI 文档](/docs/integrations/chat/azure_chat_openai)。_

假设您的部署名称是 `gpt-35-turbo-instruct-prod`。在 `openai` Python API 中，您可以使用 `engine` 参数指定此部署。例如：

```python
import openai

client = openai.AzureOpenAI(
    api_version="2023-12-01-preview",
)

response = client.completions.create(
    model="gpt-35-turbo-instruct-prod",
    prompt="Test prompt"
)
```

In [ ]:
%pip install --upgrade --quiet  langchain-openai

In [1]:
import os

os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "..."
os.environ["AZURE_OPENAI_API_KEY"] = "..."

In [2]:
# Import Azure OpenAI
from langchain_openai import AzureOpenAI

In [3]:
# Create an instance of Azure OpenAI
# Replace the deployment name with your own
llm = AzureOpenAI(
    deployment_name="gpt-35-turbo-instruct-0914",
)

In [5]:
# Run the LLM
llm.invoke("Tell me a joke")

" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

我们还可以打印 LLM 并查看其自定义打印。

In [6]:
print(llm)

AzureOpenAI
Params: {'deployment_name': 'gpt-35-turbo-instruct-0914', 'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 256}
